In [3]:
import numpy as np
import pandas as pd
import orca
import os; os.chdir('../')
import warnings; warnings.simplefilter('ignore')
from urbansim.utils import misc
import pandana as pdna
    
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
import pandas as pd
import orca as orc
# import os; os.chdir('/home/juan/ual_model_workspace/spring-2019-models/')
import warnings; warnings.simplefilter('ignore')
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import seaborn as sns

# from scripts import datasources, models, variables, utils
# scripts = '~/ual_model_workspace/spring-2019-models/scripts'

In [4]:
from scripts import datasources, models, variables, utils

Registering model step 'auto_ownership'
Registering model step 'TOD_choice'
Registering model step 'primary_mode_choice'
Registering model step 'WLCM'


In [5]:
import urbansim_templates
urbansim_templates.__version__

'0.1.1'

# Loading data

In [6]:
# orca.run(['initialize_network_small', 'initialize_network_walk','impute_missing_skims']) 

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Running step 'impute_missing_skims'
Time to execute step 'impute_missing_skims': 94.47 s
Total time to execute iteration 1 with iteration value None: 94.47 s


In [7]:
#Loading Data 
# beam_skims = orca.get_table('beam_skims').to_frame()
reset_beam_skims = beam_skims.reset_index()

students = pd.read_csv('/home/juan/ual_model_workspace/spring-2019-models/notebooks-juan/students_with_school_id.csv')
schools = pd.read_csv('/home/juan/ual_model_workspace/spring-2019-models/notebooks-juan/schools.csv').rename({"parcel_id": "school_parcel_id"}, axis=1)
# parcels = orca.get_table('parcels').to_frame()

#Preprocessing
students = students[students.AGE <= 18]


In [8]:
#Adding list of grades offered by each school
list_grades = []
for index, row in schools.loc[:,schools.columns.str.startswith("grade_")].iterrows():
    x = np.array(row)
    list_grades.append(x)
    
schools['list_grades'] = list_grades

## Defining set choice contrains

In [10]:
#Small netwrok 
nodessmall = pd.read_csv('/home/data/fall_2018/bay_area_tertiary_strongly_nodes.csv').set_index('osmid')
edgessmall = pd.read_csv('/home/data/fall_2018/bay_area_tertiary_strongly_edges.csv')
# netsmall = pdna.Network(nodessmall.x, nodessmall.y, edgessmall.u,
#                                 edgessmall.v, edgessmall[['length']],
#                                 twoway=False)
# netsmall.precompute(25000)



In [11]:
def node_id_small(x, y, netsmall):
    """ Return the node ID given a pair of coordinates"""
    idssmall = netsmall.get_node_ids(x, y)
    return idssmall


In [12]:
#Merging students and schools 
students_1 = students.merge(schools, how = 'left', on = 'school_id').drop(["parcel_id_work", "zone_id_work", "nodeID"], axis=1)
# Droping students with no assigned school
students_1 = students_1.dropna(subset=['school_id'])

In [13]:
#Define the node ID for each home and school location in students dataset
students_1['node_id_home'] = node_id_small(students_1.HXCORD, students_1.HYCORD, netsmall)
students_1['node_id_school'] = node_id_small(students_1.Longitude, students_1.Latitude, netsmall)

#Creating a df for public shcools only
df_public = students_1[students_1.school_id <= 2827].loc[:,['SAMPN', 'PERNO', 'school_id',
                                                        'AGE','HCITY','HYCORD','HXCORD',
                                                        'SNAME_lookup','SCITY_lookup', 
                                                        'node_id_home', 'node_id_school']]

# #Creating a df for private shcools only
df_private = students_1[students_1.school_id > 2827].loc[:,['SAMPN', 'PERNO', 'school_id',
                                                        'AGE','HCITY','HYCORD','HXCORD',
                                                        'SNAME_lookup','SCITY_lookup', 
                                                        'node_id_home', 'node_id_school']]

In [14]:
#Setting public and private schools as POIs. 
netsmall.set_pois('public_school', 500000, 10000, schools[schools.type == 'public']["Longitude"], 
                  schools[schools.type == 'public']["Latitude"])


netsmall.set_pois('private_school', 500000, 10000, 
                  schools[schools.type == 'private'].Longitude, 
                  schools[schools.type == 'private'].Latitude)

In [15]:
#Public schools. 95% of the time, the school is within the x's closest schools. Finding x
n =50

# n closest public schools per each node in netsmall
distance_matrix_public = netsmall.nearest_pois(200000, 
                                               'public_school', 
                                               num_pois=n,
                                               include_poi_ids=True)

# Selects POIS id's only 
public_nodes = distance_matrix_public.iloc[:,n:]

# Creates a list of n closest POIs for each node
list_values = []
for index, row in public_nodes.iterrows():
    x = np.array(row)
    x = x[~np.isnan(x)]
    list_values.append(x)

#Add created list to public nodes 
public_nodes['list_values'] = list_values


merge = df_public.merge(public_nodes.loc[:,['list_values']], how = 'left', left_on = 'node_id_home', right_index= True)

school_position = []
for index, row in merge.iterrows():
    school_position.append(np.isin(row['school_id']-1000,row['list_values'])==True)
    
merge['school_position'] = school_position

merge.school_position.mean()

0.9131504922644164

In [23]:
#Private schools. 95% of the time, the school is within the x's closest schools. Finding x
n =100

distance_matrix_private = netsmall.nearest_pois(100000, 
                                                'private_school', 
                                                num_pois=n, 
                                                include_poi_ids=True)

private_nodes = distance_matrix_private.iloc[:,n:]

list_values = []
for index, row in private_nodes.iterrows():
    x = np.array(row)
    x = x[~np.isnan(x)]
    list_values.append(x)
    
private_nodes['list_values'] = list_values


merge = df_private.merge(private_nodes.loc[:,['list_values']], how = 'left', left_on = 'node_id_home', right_index= True)

school_position = []
for index, row in merge.iterrows():
    school_position.append(np.isin(row['school_id']-1000,row['list_values'])==True)
    
merge['school_position'] = school_position

merge.school_position.mean()

0.8758465011286681

In [ ]:
# students = students[students.AGE <= 18]

In [285]:
def school_available(age, grades_offered):
    """ Checks if a school offers a grade according to the age range
    Input: Age: int in range (5-18)
           grades_offered: 13 element array
    Output: True if grade if offered acoording to age, false otherwise. 
     """
    index = age - 6
    
    if index < 0:
        index = 0

    if index < 12:
        result = (grades_offered[index] == 1) | (grades_offered[index + 1] == 1)
        
    elif index == 12:
        result = (grades_offered[index] == 1)
    
    return result

In [289]:
def school_choice_set(house_node_id, kid_age):
    """ Determines the school choice set given home node id and age of the student (4-18)
    Output: Pandas series with available school IDs"""
    
    public_id = distance_matrix_public.iloc[:,50:].loc[house_node_id] + 1000
    private_id = distance_matrix_private.iloc[:,100:].loc[house_node_id] + 1000

    schools_filter = pd.concat([public_id, private_id])
    schools_set = schools[schools.school_id.isin(schools_filter)]
    school_availability = [school_available(kid_age, x) for x in schools_set.list_grades]
    try:
        schools_available = schools_set[school_availability].school_id
        return schools_available
    except Exception as e:
        print(e)
        print("empty dataframe?")
    
    return np.nan

In [290]:
def get_zone_id(parcel_id):
    '''gets the zone_id (TAZ) of school and home locations by the parcel id'''
    try: 
        parcel_id = (parcels.iloc[parcel_id])["zone_id"]

    except Exception as e:
#         print(e)
        parcel_id = np.nan
    return parcel_id
 

### Create parameters table with distances and generalized cost

In [291]:
stu = students_1[["node_id_home", "AGE", "school_parcel_id", "parcel_id_home"]] 
for index, record in stu.iterrows():
    age = int(record["AGE"])
    parcel_id_home = int(record["parcel_id_home"])
    node_id_home = int(record["node_id_home"])
    school_choices = school_choice_set(node_id_home, age)
    school_parcel_id = [schools[schools["school_id"] == choice]["school_parcel_id"].values[0] for choice in school_choices]
    df = pd.DataFrame({"node_id_home": node_id_home, "parcel_id_home": parcel_id_home, "age": age,
                       "school_choice": school_choices, "parcel_id_school": school_parcel_id})
    df["school_zone_id"] = df["parcel_id_school"].apply(get_zone_id)
    df["home_zone_id"] = df["parcel_id_home"].apply(get_zone_id)

    
    dfs.append(df)
    
all_choices = pd.concat(dfs, axis=0)


In [293]:
all_choices_costs = pd.merge(all_choices, reset_beam_skims,  how='left', left_on=['school_zone_id','home_zone_id'], right_on = ['to_zone_id','from_zone_id'])
all_choices_costs = all_choices_costs.set_index("node_id_home")
all_choices_costs.head()
all_choices_costs.to_csv("all_choices_costs.csv")


In [292]:
print(len(all_choices))
print(len(all_choices_costs))

291179
35


In [309]:
all_choices_costs

,parcel_id_home,age,school_choice,parcel_id_school,school_zone_id,home_zone_id,from_zone_id,to_zone_id,dist,gen_cost_BIKE,...,gen_cost_WALK,gen_cost_WALK_TRANSIT,gen_tt_BIKE,gen_tt_CAR,gen_tt_DRIVE_TRANSIT,gen_tt_RIDE_HAIL,gen_tt_RIDE_HAIL_POOLED,gen_tt_RIDE_HAIL_TRANSIT,gen_tt_WALK,gen_tt_WALK_TRANSIT
node_id_home,,,,,,,,,,,,,,,,,,,,,
65497078,1195607,17,2021,1195135,901.0,742.0,742.0,901.0,36161.8698,46.519338,...,5338.670369,2064.891192,150.786537,32.394499,53.645277,58.084325,45.132109,51.227852,635.884119,2156.841723
65497078,1195607,17,2093,1171860,1273.0,742.0,742.0,1273.0,89382.7436,114.983713,...,13195.805621,5103.874356,372.705131,80.070782,132.597183,143.569354,111.554843,126.621935,1571.740275,5331.152172
65497078,1195607,17,2094,1186767,1085.0,742.0,742.0,1085.0,65387.4842,84.115741,...,9653.323413,3733.712911,272.650512,58.575367,97.000784,105.027419,81.607369,92.629622,1149.798476,3899.976823
65497078,1195607,17,2095,1195560,883.0,742.0,742.0,883.0,32782.2558,42.171736,...,4839.729213,1871.910706,136.694338,29.366976,48.631700,52.655883,40.914155,46.440202,576.455697,1955.267730
65497078,1195607,17,2097,1192891,761.0,742.0,742.0,761.0,26087.4014,33.559344,...,3851.350542,1489.625555,108.778361,23.369596,38.700042,41.902399,32.558589,36.956096,458.730822,1555.959249
65497078,1195607,17,2409,1299756,956.0,742.0,742.0,956.0,41424.4116,53.289175,...,6115.593026,2365.389376,172.730106,37.108785,61.452133,66.537184,51.700066,58.682906,728.422663,2470.721226
65497078,1195607,17,2420,1312872,340.0,742.0,742.0,340.0,61557.2550,79.188459,...,9087.856770,3515.001695,256.679353,55.144174,91.318730,98.875186,76.827021,87.203619,1082.446264,3671.526298
65497078,1195607,17,3240,1183308,1290.0,742.0,742.0,1290.0,75317.1120,96.889409,...,11119.260048,4300.707956,314.054737,67.470518,111.731152,120.976698,94.000120,106.696193,1324.404841,4492.220412
65497078,1195607,17,3255,1183308,1290.0,742.0,742.0,1290.0,75317.1120,96.889409,...,11119.260048,4300.707956,314.054737,67.470518,111.731152,120.976698,94.000120,106.696193,1324.404841,4492.220412


In [294]:
# for table_name in orca.list_tables():
#     print(table_name.upper())
#     print(orca.get_table(table_name).to_frame().columns.tolist())
#     print()

# Additional code

In [295]:
# #Creating school availability matrix based on distance
# school_availability_distance = pd.DataFrame(nodessmall.index)
# for x in range(1000,3547,1):
#     name = "school_" + str(x)
#     school_availability_distance[name] = x
# school_availability_distance = school_availability_distance.set_index('osmid')

In [296]:
# school_availability_distance

In [297]:

# a = students.sample(10)
# a.loc[:,['AGE','HCITY','HYCORD','HXCORD', 'school_id']]

In [298]:
# students[(students.SAMPN == 7176467) & (students.PERNO ==4) ][['HCITY', 'HYCORD','HXCORD',
#                                                                 'AGE','SNAME_lookup','SCITY_lookup', 'school_id']]

In [299]:
# schools[schools.school_id == 1271]

In [300]:
# d[d.type == 'public'].sort_values('dist', ascending=False)

In [301]:
# os.listdir("/home/data/fall_2018/")

In [302]:
# # Estimating 95th percentail of distances between home and school
# a = schools.merge(parcels[['zone_id']], left_on ='parcel_id', right_index=True)
# b = students[['SAMPN','PERNO','zone_id_home', 'school_id']].merge(a[['zone_id','school_id','type']], on = 'school_id')
# c = beam_skims.reset_index(level=['from_zone_id', 'to_zone_id'])
# d = b.merge(c.loc[:,['from_zone_id','to_zone_id', 'dist']], 
#             left_on = ['zone_id_home','zone_id'], 
#             right_on = ['from_zone_id', 'to_zone_id'])

# public_dist_limit = d[d.type == 'public'].quantile(1).dist
# private_dist_limit = d[d.type == 'private'].quantile(1).dist

# sns.distplot(d[d.type == 'public'].dist,kde=False)
# sns.distplot(d[d.type == 'private'].dist,kde=False);
# d.sort_values('dist', ascending=False)

In [303]:
# a = students.loc[:,['HCITY','SNAME_lookup', 'SCITY_lookup', 'SZIP_lookup',
#                 'SXCORD_lookup', 'SYCORD_lookup', 'SPrimaryCity_lookup',]]
# same_city = []
# for index, row in a.iterrows():
#     if row['HCITY'].lower() == row['SCITY_lookup']:
#         same_city.append(True)
#     else:
#         same_city.append(False)

# a['same_city'] = same_city
# a.same_city.mean()

In [304]:
#'network_aggregations_small'])
# orca.run(['initialize_network_walk', 'network_aggregations_walk'])

# orca.get_table('nodeswalk').to_frame().to_csv('./data/walk_net_vars.csv')
# orca.get_table('nodessmall').to_frame().to_csv('./data/drive_net_vars.csv')
# orca.get_table('zones').to_frame().to_csv('./data/zones_w_access_vars-2025-b-lt.csv')

In [305]:
# os.listdir("/home/data/spring_2019/2025")

In [306]:
# a = students.loc[:,['SAMPN', 'PERNO', 'AGE', 'HPrimaryCity','school_id']].merge(schools.loc[:,['school_id','City','District']], how = 'inner', on = 'school_id')
# students[students.SAMPN == 1342402].loc[:,['SNAME_lookup','school_id', 'HPrimaryCity']]
# schools[schools.school_id == 1710]